In [1]:
import os
import sys
from pathlib import Path

import pendulum
import numpy as np
import porepy as pp


sys.path.extend([
    '/home/haakon/mastersproject', 
    '/home/haakon/mastersproject/src/mastersproject'
])

import GTS as gts
import GTS.test.util as test_util



/home/haakon/porepy/src/porepy/viz/exporter.py:928: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  angle = np.arccos(np.dot(normals[:, fi], reference))


## Setup
We consider a simple setup in 3D with one vertical fracture in the yz-plane.

#### Scalar:
* Boundary condition:
    * Dirichlet: 0
* Source terms:
    * No source
#### Mechanics:
* Boundary condition:
    *  
* Source terms:
    *

In [8]:
def setup_simple_gb(**kwargs):
    """ Setup simple grid bucket
    
    Keyword arguments used for
        length_scale,
        scalar_scale,
        path_tail (Default: "test_1"),
        setup_loggers (Default: False),
        
    """
    length_scale = kwargs.get("length_scale", 1)
    scalar_scale = kwargs.get("scalar_scale", 1)
    
    # Storage path details
    path_tail = kwargs.get("path_tail", "test_1")
    name = "test_parameter_scaling_on_regular_grid_ipynb"
    date = pendulum.now().format("YYMMDD")
    _folder_root = f"{name}/{date}/{path_tail}/ss{scalar_scale}ls{length_scale}"
    
    # Parameter setup
    
    # Size of domain - scaled by length scale.
    physdims = np.array([10, 10, 10]) / length_scale

    params = {
            "mesh_args":
                {None},
            "bounding_box":
                {"xmin": 0, "xmax": physdims[0],
                 "ymin": 0, "ymax": physdims[1],
                 "zmin": 0, "zmax": physdims[2],},
            "shearzone_names":
                ["S1_1"],
            "length_scale":
                length_scale,
            "scalar_scale":
                scalar_scale,
            # Turn of gravity effects:
            "_gravity_bc_p": False,
            "_gravity_src": False,
            "_gravity_bc": False,
        }
    
    setup_loggers = kwargs.get("setup_loggers", False)

    setup = test_util.prepare_setup(
            model=gts.ContactMechanicsBiotISC,
            path_head=_folder_root,
            params=params,
            prepare_simulation=False,
            setup_loggers=setup_loggers,
        )
    
    ### Set up simple grid bucket with one fracture ###
    
    # Grid is 2x2x2 cells, with a fracture in the lower middle part of the grid.
    #  _ _
    # |   |
    # |_|_|

    # Frac point coordinates, scaled by length scale
    # The fracture lies in the yz-plane.
    f_1 = np.array([
        [5, 5, 5, 5],
        [0, 0, 5, 5],
        [0, 5, 5, 0]

    ]) / length_scale
    
    # Number of cells in each dimension.
    n_cells = np.array([2, 2, 2])

    gb = pp.meshing.cart_grid(
        fracs=[f_1],
        nx=n_cells,
        physdims=physdims,
    )
    
    # Assign the simple grid to the grid bucket
    setup.set_grid(gb)
    
    run_model = kwargs.get("run_model", True)
    if run_model:
        nl_options = {
            "max_iterations": 20,
        }
        pp.run_time_dependent_model(setup, nl_options)
    
    return setup

# Run with unscaled variables
I.e., the following (default) scalings apply:\
`length_scale = 1`\
`scalar_scale = 1`

In [9]:
unscl = setup_simple_gb()

No injection cell was tagged.
/home/haakon/.local/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


# Next, re-run with scaled variables
We apply the following scalings:\
`length_scale = 1`\
`scalar_scale = 1 * pp.MEGA`

This isolates testing for the `scalar_scale`.

In [10]:
scl = setup_simple_gb(
    length_scale=1,
    scalar_scale=1*pp.MEGA,
)

/home/haakon/.local/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


# Compare the results

Recall, we only scale the pressure scale here

In [11]:
def assert_solutions(s1, s2, dim):
    """ Assert that solutions scale correctly
    
    Parameters
    ----------
    s1, s2 : ContactMechanicsBiotISC
        Setups
    dim : int
        which grid dimension to extract solutions over
    """
    
    def fetch_values_from_setup(s, dim):
        # scales
        ls = s.length_scale
        ss = s.scalar_scale
        # grids
        gb = s.gb
        g = gb.grids_of_dimension(dim)[0]
        # data
        d = gb.node_props(g)
        state = d[pp.STATE]
        
        ### Quantities ###
        cell_centers = g.cell_centers * ls
        cell_volumes = g.cell_volumes * (ls)**dim
        face_areas = g.face_areas * (ls)**(dim-1)
        p = state['p'] * ss
        p_exp = state['p_exp']
        
        Nd = s.Nd
        # Matrix displacement
        if dim == Nd:
            u = state['u']
            traction = state['traction_exp']
            tangential_traction = traction[:-1, :]
            normal_traction = traction[-1, :]
        # Fracture displacement in global coordinates
        elif dim == Nd - 1:
            e, d_e = next(gb.edges())
            # Displacement
            mg = d_e['mortar_grid']
            mortar_u = d_e['state']['mortar_u']
            u = (mg.mortar_to_slave_avg(nd=Nd) 
                 * mg.sign_of_mortar_sides(nd=Nd)
                 * mortar_u)
            
            # Traction
            # recall: traction is represented in local coordinates directly.
            proj = d_e['tangential_normal_projection']
            proj.tangential_basis[:,:,0]
            traction = d['state']['contact_traction']
            tangential_traction_local = traction[:-1]
            normal_traction = traction[-1]
            tangential_traction = proj.tangential_basis[:,:,0].dot(tangential_traction_local)
            # upscale the scaled result
            normal_traction = normal_traction * ss * ls**(Nd-1)
            tangential_traction = tangential_traction * ss * ls**(Nd-1)

        u = u * ls
        
        return cell_centers, cell_volumes, face_areas, p, p_exp, u, normal_traction, tangential_traction
    
    (cell_centers1, cell_volumes1, face_areas1, p1, p_exp1, u1, 
     normal_traction1, tangential_traction1) = fetch_values_from_setup(s1, dim)
    
    (cell_centers2, cell_volumes2, face_areas2, p2, p_exp2, u2, 
     normal_traction2, tangential_traction2) = fetch_values_from_setup(s2, dim)

    ### Assertions ###
    # Assert that each geometric index corresponds to one another
    assert np.allclose(cell_centers1, cell_centers2)
    
    # Assert that all volumes and areas are the same
    assert np.allclose(cell_volumes1, cell_volumes2)
    assert np.allclose(face_areas1, face_areas2)
    
    ## Non-zero on all grids ##
    # Assert that the pressure results are equal
    p_true = np.allclose(p1, p2)
#     assert p_true
    print(f"p: {p_true}")
    
    # Assert that the exported results are upscaled correctly
    p_exp_true = np.allclose(p_exp1, p_exp2)
#     assert p_exp_true
    print(f"p_exp: {p_exp_true}")
    
    # Assert that the displacement results are equal
    u_true = np.allclose(u1, u2)
#     assert u_true
    print(f"u: {u_true}")


    # Traction
    n_traction_true = np.allclose(normal_traction1, normal_traction2)
#     assert n_traction_true
    print(f"normal_traction: {n_traction_true}")
    t_traction_true = np.allclose(tangential_traction1, tangential_traction2)
#     assert t_traction_true
    print(f"tangential_traction: {t_traction_true}")
    
    assert p_true and p_exp_true and u_true and n_traction_true and t_traction_true

In [12]:
# ONLY SCALAR SCALE

## Assert solutions on 3D grid ##
assert_solutions(scl, unscl, 3)

## Assert solutions on 2D grid ##
assert_solutions(scl, unscl, 2)

p: True
p_exp: True
u: True
normal_traction: True
tangential_traction: True
p: True
p_exp: True
u: True
normal_traction: True
tangential_traction: True


In [84]:
def check_parameter_scaling(s1, s2, dim):
    """ Test parameter scaling for two setups"""
    
    def fetch_values_from_setup(s, dim):
        # scales
        ls = s.length_scale
        ss = s.scalar_scale
        # grids
        gb = s.gb
        g = gb.grids_of_dimension(dim)[0]
        # data
        data = gb.node_props(g)
        params_u = data['parameters']['mechanics']
        params_p = data['parameters']['flow']
        
        neu_p = params_p['bc'].is_neu
        dir_p = params_p['bc'].is_dir
        
        # Compute unscaled quantities: 
        # MECHANICS
        if dim == s.Nd: # On rock matrix
            neu_u = (params_u['bc'].is_neu).ravel(order='F')
            dir_u = (params_u['bc'].is_dir).ravel(order='F')
            mech = {
                'bc_values': {
                    'dir': params_u['bc_values'][dir_u].reshape((3, -1), order='F') * ls, # [L]
                    'neu': params_u['bc_values'][neu_u].reshape((3, -1), order='F') * ss * ls**(dim-1), # [P L^(dim-1)]
                },
                'source':
                    params_u['source'] * ss * ls**(dim-1), # [P L^(dim-1)]
                'biot_alpha': # [ ]
                    params_u['biot_alpha'], 
                'mu':
                    params_u['fourth_order_tensor'].mu * ss, # [P]
                'lambda':
                    params_u['fourth_order_tensor'].lmbda * ss, # [P]
            }
        elif dim == s.Nd-1: # On fracture
            mech = params_u # We're only interested in the friction coefficient
        
        # FLOW
        flow = {
            'bc_values': {
                'dir': params_p['bc_values'][dir_p] * ss, # [P]
                'neu': params_p['bc_values'][neu_p] * ls**dim, # [L^(dim)]
            },
            'mass_weight':  
                params_p['mass_weight'] / ss,  # [1/P]
            'source':
                params_p['source'] * ls**dim, # [L^(dim)]
            'biot_alpha': # [ ]
                params_p['biot_alpha'],
            'kxx':
                params_p['second_order_tensor'].values[0, 0, :] * ls**2 / ss, #k/mu: [L^2 / P]
        }
        return mech, flow
    
    mech1, flow1 = fetch_values_from_setup(s1, dim)
    mech2, flow2 = fetch_values_from_setup(s2, dim)
    
    # -- ASSERTIONS --
    # Mechanics
    if dim == s1.Nd:
        # BC:
        assert np.allclose(mech1['bc_values']['dir'], mech2['bc_values']['dir'])
        assert np.allclose(mech1['bc_values']['neu'], mech2['bc_values']['neu'])
        # Source
        assert np.allclose(mech1['source'], mech2['source'])
        # Physical parameters
        assert np.allclose(mech1['biot_alpha'], mech2['biot_alpha'])
        assert np.allclose(mech1['mu'], mech2['mu'])
        assert np.allclose(mech1['lambda'], mech2['lambda'])
    elif dim == s1.Nd-1:
        assert np.allclose(mech1['friction_coefficient'], mech2['friction_coefficient'])
    
    # Flow
    # BC:
    assert np.allclose(flow1['bc_values']['dir'], flow2['bc_values']['dir'])
    assert np.allclose(flow1['bc_values']['neu'], flow2['bc_values']['neu'])
    # Source
    assert np.allclose(flow1['source'], flow2['source'])
    # Physical parameters
    assert np.allclose(flow1['biot_alpha'], flow2['biot_alpha'])
    assert np.allclose(flow1['kxx'], flow2['kxx'])
    
def check_mortar_parameter_scaling(s1, s2):
    def fetch_values_from_setup(s, dim):
        # scales
        ls = s.length_scale
        ss = s.scalar_scale
        # grids
        gb = s.gb
        
        for e, d in gb.edges():
            k = d['parameters']['flow']['normal_diffusivity']

In [85]:
check_mortar_parameter_scaling(scl, unscl)

In [120]:
for e, d in scl.gb.edges():
    print(d['discretization_matrices']['flow']['Robin_discr'].toarray() * scl.scalar_scale**2)
    d=d

[[-243887.11514382       0.        ]
 [      0.         -243887.11514382]]


In [113]:
d['discretization_matrices']['flow']['Robin_discr'].toarray()

array([[-2.43887115e-07,  0.00000000e+00],
       [ 0.00000000e+00, -2.43887115e-07]])

In [118]:
for e, d in unscl.gb.edges():
    print(d['discretization_matrices']['flow']['Robin_discr'].toarray())

[[-243887.11514382       0.        ]
 [      0.         -243887.11514382]]


In [102]:
d['state']['mortar_p']

array([6.14713941e-11, 6.95372972e-11])

In [95]:
d['parameters']['flow']['normal_diffusivity']

array([164010.3044247, 164010.3044247])

In [44]:
# ONLY SCALAR SCALE

## Assert parameters on 3D grid ##
check_parameter_scaling(scl, unscl, 3)

## Assert parameters on 2D grid ##
check_parameter_scaling(scl, unscl, 2)

### Sjekk slip i globale koordinater!

* Sjekk med mindre skalering (1e8, 1e6)
* Introduser lengdeskalering


* Du kan regne ut analytisk om du vil ha slip
Hvilke skjærkrefter har du. Bruk MC for å regne om du får slip.

Du kan regne betingelser for å akkurat ha slip eller akkurat ikke ha slip.

Rotasjon av stress kan gjøre at vi får slip.

# What do we observe?
We observe that the displacement of the fracture for each of the runs is different. Not but a big amount, but still different. We investigate reasons why this can be.

1. The numerical parameter `c` (`contact_mechanics_numerical_parameter`) may have units, and therefore influence the results. We first investigate how this parameter is used.

In the code, `c` is used in `contact_conditions.py`. 
It is used to compute the friction bound in the following way:
```
friction_bound = (
    friction_coefficient * np.clip(
        - contact_force_normal 
        + c_num_normal * displacement_jump_normal, 
    0, np.inf) 
    + scaled_cohesion
```

where `friction_coefficient` is the assigned (unitless) friction coefficient, `np.clip` is in this instance a function that clips and output of the inner expression in the range `[0, +inf]`; `contact_force_normal` is the normal contact force from the previous iteration (hence, it is scaled) and has units `[Pa m2]` (tractions are scaled with area, according to the code); `c_num_normal` is the product `c_num * mean_constit * area` where `c_num` the numerical parameter `c`, `mean_constit` is the average of `mu` and `lambda` (`[Pa]`, scaled Lame parameters) mapped from the nearest grid cell to internal boundary the mortar grid to the fracture (by averaging), `area` `[m2]` is the scaled area of the fracture cell; `displacement_jump_normal` `[m]` is the fracture displacement normal dilation; `scaled_cohesion` is the `cohesion` (0 by default) multiplied with fracture area `[m2]`.

------------
# Compare moderately scaled solutions
We apply the following scalings:\
`length_scale = 1`\
`scalar_scale = 1e5`

We aim to compare the solutions with scalar scale 1e5 and 1e6. Hopefully these should be very similar.

In [45]:
scl_mod = setup_simple_gb(
    length_scale=1,
    scalar_scale=1e6,
)

/home/haakon/.local/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [47]:
# ONLY SCALAR SCALE

## Assert solutions on 3D grid ##
print("dim: 3")
assert_solutions(scl, scl_mod, 3)

## Assert solutions on 2D grid ##
print("dim: 2")
assert_solutions(scl, scl_mod, 2)

dim: 3
p: True
p_exp: True
u: True
normal_traction: True
tangential_traction: True
dim: 2
p: True
p_exp: True
u: True
normal_traction: True
tangential_traction: True


In [46]:
## Assert parameters on 3D grid ##
check_parameter_scaling(scl, scl_mod, 3)

## Assert parameters on 2D grid ##
check_parameter_scaling(scl, scl_mod, 2)

------------
# Apply length scale in addition
We apply the following scalings:\
`length_scale = 100`\
`scalar_scale = pp.MEGA`

In [48]:
scl_l1 = setup_simple_gb(
    length_scale=.1,
    scalar_scale=pp.MEGA,
)

/home/haakon/.local/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [49]:
scl_l2 = setup_simple_gb(
    length_scale=.01,
    scalar_scale=pp.MEGA,
)

/home/haakon/.local/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [50]:
## Assert parameters on 3D grid ##
check_parameter_scaling(scl_l1, scl_l2, 3)

## Assert parameters on 2D grid ##
check_parameter_scaling(scl_l1, scl_l2, 2)

In [51]:
## Assert solutions on 3D grid ##
assert_solutions(scl_l1, scl_l2, 3)

p: False
p_exp: False
u: False
normal_traction: True
tangential_traction: True


AssertionError: 

In [52]:
# We observe discreprancies 
%debug

> <ipython-input-11-053b18d4a322>(102)assert_solutions()
     98     t_traction_true = np.allclose(tangential_traction1, tangential_traction2)
     99 #     assert t_traction_true
    100     print(f"tangential_traction: {t_traction_true}")
    101 
--> 102     assert p_true and p_exp_true and u_true and n_traction_true and t_traction_true

ipdb> (p1-p2)
array([  9509.43203906, -79935.6754485 ,  22552.47472484,  18491.05820819,
        39356.48409328, -55076.15488598,   3629.70316187,   9535.34945661])
ipdb> (p1-p2)/p2
array([ 0.00133404, -0.01243743,  0.0031233 ,  0.00273505,  0.00636189,
       -0.00875856,  0.00058205,  0.00154501])
ipdb> q


In [53]:
## Assert solutions on 2D grid ##
assert_solutions(scl_l1, scl_l2, 2)

p: False
p_exp: False
u: False
normal_traction: False
tangential_traction: False


AssertionError: 

In [54]:
%debug

> <ipython-input-11-053b18d4a322>(102)assert_solutions()
     98     t_traction_true = np.allclose(tangential_traction1, tangential_traction2)
     99 #     assert t_traction_true
    100     print(f"tangential_traction: {t_traction_true}")
    101 
--> 102     assert p_true and p_exp_true and u_true and n_traction_true and t_traction_true

ipdb> (p1-p2)/p2
array([-0.60207816])
ipdb> (tangential_traction1-tangential_traction2)/tangential_traction1


/home/haakon/anaconda3/envs/pore/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([       nan, 0.35701384, 0.36453701])
ipdb> q


In [55]:
A, b = scl_l1.assembler.assemble_matrix_rhs()

In [79]:
f"{np.max(np.abs(A)):.4e}"

'9.8029e+07'

In [80]:
f"{np.min(np.abs(A)):.4e}"

'0.0000e+00'

In [81]:
f"{np.max(np.sum(np.abs(A), axis=1)):.4e}, {np.min(np.sum(np.abs(A), axis=1)):.4e}"

'9.8029e+07, 2.0000e+00'

In [77]:
f"{np.max(np.sum(np.abs(A), axis=1)) / np.min(np.sum(np.abs(A), axis=1)):.4e}"

'4.9015e+07'

# Mulige problem (fra veilednignsmøte 24.03.2020)

* Kan hende det ikke er lengdeskalering på randkrav
* Slå av gravitasjonseffekter (sett til 0)
   - Slå av vann (Dirichlet=0) og gravitasjon.
   - Problemet kan være drevet av anisotrop bakgrunnsstress.
  
Merk: Problemet er nok ikke drevet av væske. Hvis det er satt opp riktig vil det være en likevekt mellom randkrav og kildeledd. Dermed vil ikke det være utgangspunktet for feilene våre.

Merk2: Feilen i trykket er for stort til at det handler om dårlig kondisjonering av problemet. Det er heller drevet av feil i grunnleggende skalering av -- rand, kilde, parameter.

In [ ]:
A, b = scl_l1.assembler.assemble_matrix_rhs(add_matrices=False)

In [ ]:
A

In [ ]:
key = 'mass_p'
print(A[key].toarray())
print(f"{np.max(np.sum(np.abs(A[key]), axis=0)):.4e}")

------------

In [ ]:
A2, b2 = scl_l2.assembler.assemble_matrix_rhs(add_matrices=False)

In [ ]:
key = 'mass_p'
print(A2[key].toarray())
print(f"{np.max(np.sum(np.abs(A2[key]), axis=0)):.4e}")